<img src="PillowLogo.jpg" width="400">

# Hands-On Lab: Home Price Estimation

## Introduction
In this lab, PACCAR is entering the real estate market and taking on Zillow! We will be estimating home prices, showing how good data and good science get us the best recommendations. Specifically, we'll walk through:
* A simple linear regression
* AutoML
* Data understanding & feature engineering

In [98]:
# Import any packages we will need to use for the analysis
import pandas as pd
import numpy as np
import copy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

## Import Data
When working in code, we first need to import our data. This can come from databases, spreadsheets, websites, etc.

```
[!] It is important to remember that data is loaded into your computer's memory. When people talk about "Big Data", it is commonly describing datasets that don't fit into memory!
```

In [3]:
# Read in our dataset and investigate the first several rows
home_data = pd.read_csv("../Data/Train.csv")
home_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


We should start by getting familiar with the dataset. Some common questions (and answers):

In [4]:
# What is our target variable?
print("SalePrice")
home_data.SalePrice.head(3)

SalePrice


0    208500
1    181500
2    223500
Name: SalePrice, dtype: int64

In [5]:
# How many rows of data do we have?
print('Number of rows: {}'.format(len(home_data)))

Number of rows: 1460


In [6]:
# How many attributes do we have for each row of data? What are the features?
print('Number of columns: {}'.format(home_data.shape[1]))
print(list(home_data))

Number of columns: 81
['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'

Thankfully, the city of Ames, Iowa does a terrific job of cataloging data. Have a look at *./Documentation/data_description.txt* for more information about what these inputs mean. Unfortunately, this is a rare treat but a state we should strive for.

In [7]:
# What data types are the features?
home_data.columns.to_series().groupby(home_data.dtypes).groups

{dtype('int64'): Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
        'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
        'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
        'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
        'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
        'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
       dtype='object'),
 dtype('float64'): Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object'),
 dtype('O'): Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
        'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
        'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
        'BsmtQual', 

## Measuring Quality
A strong suite of validation measures is critical to the success of an ML deployment.

For home pricing, we will use RMSE- Root Mean Square Error. 
<img src="rmse.jpg" width="400">
RMSE is commonly used in regression. We often communicate MAE (Mean Absolute Error), as it is more intuitive, but RMSE generally makes a better measure for maximizing model performance. The reason why is in the square term. RMSE penalizes larger errors more severely than smaller errors. 

For home pricing, we apply a small variation on RMSE, which is to measure the difference in the Logs of the prices. This is to assign equal weight to lower and higher priced homes.

```
[!] Plot Twist: This dataset comes from a public competition, for which there is a leaderboard! Throughout the lab, we will compare our performance against the leaderboard.
```

In [113]:
# Define RMSE formula to be used later
def getRMSE(data, target, prediction):
    data['num'] = (np.log(data[prediction]) - np.log(data[target]))**2
    RMSE = np.sqrt(sum(data.num) / len(data))
    RMSE
    return(RMSE)

# getRMSE(y_score, 'SalePrice', 'Pred_clf')

0.16836011593507821

In [112]:
# Import leaderboard
leaderboard = pd.read_csv("../Data/house-prices-advanced-regression-techniques-publicleaderboard.csv",
                          sep='\t',
                          encoding = "ISO-8859-1")

leaderboard.head()

,TeamId TeamName SubmissionDate Score
0,439244\tDSXL\t4/18/2017 9:57\t0.06626
1,439244\tDSXL\t3/23/2017 13:29\t0.06627
2,2027397\tIgor S\t9/5/2018 18:26\t0.06946
3,1958420\tZheng Pan\t8/12/2018 12:11\t0.08021
4,509205\tmohammed khamis\t9/2/2018 5:14\t0.10567


## Prep Data for Modeling
When building predictive models, it is crucial that we split our data into *train* and *test* sets. This keeps our models honest as they are trained on one set of data, then tested on another. Later, we will talk about related concepts of overfitting and target leakage in relationship to these splits.

<img src="train_test_split.svg" width="600">

In [68]:
# Split our data columns into features and target tables
X = home_data.drop(columns=['SalePrice', 'Id'])
y = home_data[['SalePrice']].astype('float')

In [69]:
# Then split the rows into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1984)

print("Training Rows: {} \nTest Rows: {}".format(len(X_train), len(X_test)))

Training Rows: 1022 
Test Rows: 438


## Linear Model
Our first approach will be a linear regression fit of home prices. This is similar to the type of fit we would perform in Minitab. Linear models are an excellent place to start due to their interpretability- we know exactly how predictions are being generated. 

On the other hand, linear models assume linear fits... an assumption that must be tested! Below we see an example of Anscombe's Quartet. From Wikipedia: 
*The data sets in the Anscombe's quartet are designed to have approximately the same linear regression line (as well as nearly identical means, standard deviations, and correlations) but are graphically very different. This illustrates the pitfalls of relying solely on a fitted model to understand the relationship between variables.*

Only the top-left quartet would be considered an appropriate model! 

<img src="AnscombesQuarter.png" width="600">

In [70]:
# One-hot encode any categorical variables
def one_hot_mixed_df(data):
    data_cat = data.select_dtypes(include='object')
    data_num = data.select_dtypes(exclude='object')
    ## One-hot transform the objects
    data_cat_dummies = pd.get_dummies(data_cat,drop_first=True)
    ## Join dummies with numerics
    data_lm = pd.concat([data_cat_dummies, data_num], axis=1, sort=False)
    return(data_lm)

# Apply transformations
X_lm = one_hot_mixed_df(X)

## Verify results
print('Total Columns: {}'.format(X.shape[1]))
print('Total Columns post-Dummy: {}'.format(X_lm.shape[1]))

Total Columns: 79
Total Columns post-Dummy: 245


In [71]:
print("Shape prior to NA cleanup: {}".format(X_lm.shape))

# NA values will crash the model fits, so let's see how to handle them
def count_NA(data):
    na_counts = data.isnull().sum(axis=0).reset_index(name='NACount')
    ## Print out the list of columns with missing values
    return(na_counts[na_counts.NACount>0])
    
print(count_NA(X_lm))

## We'll make a simplifying assumption that we can drop lot frontage and garage year built features. 
X_lm.drop(columns=['LotFrontage', 'GarageYrBlt'], inplace=True)

## For MasVnrArea, we will drop the NA rows. Note that rows also need to be dropped from our target!!!
row_drop_index =X_lm.MasVnrArea.notna()
X_lm = X_lm[row_drop_index]
y_lm = y[row_drop_index]

print("Shape of train after NA cleanup: {}".format(X_lm.shape))

Shape prior to NA cleanup: (1460, 245)
           index  NACount
210  LotFrontage      259
216   MasVnrArea        8
233  GarageYrBlt       81
Shape of train after NA cleanup: (1452, 243)


In [72]:
# Then split the rows into train and test
X_train, X_test, y_train, y_test = train_test_split(X_lm, y_lm, test_size=0.3, random_state=1984)

Let's finally fit the linear model to an object called "reg", then look at some information about the model.

In [73]:
# Fit a linear model
reg = LinearRegression().fit(X_train, y_train)

# Print summaries
print("Score (Training):")
print(reg.score(X_train, y_train))
print("------------------------------------------------------------------")
print("Score (Test):")
print(reg.score(X_test, y_test))
print("------------------------------------------------------------------")
print("Intercept:")
print(reg.intercept_)
print("------------------------------------------------------------------")
print("Coefficients:")
print(reg.coef_)

Score (Training):
0.9359801295370493
------------------------------------------------------------------
Score (Test):
0.7649615984547558
------------------------------------------------------------------
Intercept:
[672744.52108104]
------------------------------------------------------------------
Coefficients:
[[ 4.60282620e+04  3.39292627e+04  4.16763435e+04  3.56334015e+04
   5.95591659e+03  1.65453760e+03  3.85229628e+03 -6.31798182e+03
   2.50427842e+03  1.03383998e+04 -8.14880006e+03  5.38988852e+03
  -3.31392974e+04  7.92224184e+03 -5.35805941e+03 -7.21267325e+03
  -1.98475989e+03  8.60986970e+03 -1.68426653e+04  1.78152167e+04
   1.61945799e+04  4.91470188e+03 -5.72470053e+03 -7.22758685e+02
   2.65165900e+04 -1.03733602e+04 -1.50497133e+03 -3.38956999e+03
   9.07483624e+03 -1.73439799e+04 -7.39768722e+03  2.28250094e+04
  -6.86783751e+03  3.68415973e+04  2.62649299e+04 -2.56614191e+03
   5.81746274e+03 -3.50518625e+03  7.05920125e+03  1.48745750e+04
   3.78278720e+04 -6.28518

Pause and reflect on a few questions:
* Why is our training score so much higher than testing?
* How might we improve our scores with a linear model?
    * Look at leverage and QQ-plots to identify points with high leverage and non-normal distributions. Feature selection and dimension reduction. 
* If we have a midling model, would you rely on the model weights to drive insights? Think about Anscombe's Quartet.

In [74]:
clf = Ridge(alpha=1)
clf.fit(X_train, y_train)

# Print summaries
print("Score (Training):")
print(clf.score(X_train, y_train))
print("------------------------------------------------------------------")
print("Score (Test):")
print(clf.score(X_test, y_test))
print("------------------------------------------------------------------")
print("Intercept:")
print(clf.intercept_)
print("------------------------------------------------------------------")
print("Coefficients:")
print(clf.coef_)

Score (Training):
0.9300822750791763
------------------------------------------------------------------
Score (Test):
0.7753468093984774
------------------------------------------------------------------
Intercept:
[170429.00067665]
------------------------------------------------------------------
Coefficients:
[[ 1.99043112e+04  9.18688610e+03  1.79800663e+04  1.38107772e+04
   1.17992671e+04  2.81294973e+02  7.93899766e+03  1.37798174e+03
   6.16887326e+02  1.29839258e+04 -4.19725051e+03  6.93200471e+03
  -1.16614615e+04  7.10492381e+03 -5.31679067e+03 -5.38459269e+03
  -1.98756148e+03  6.74321918e+03  2.87656577e+03  3.29089658e+03
   8.90547853e+03  2.80071382e+03 -1.05417649e+04 -3.75824896e+03
   1.88738963e+04 -1.38520069e+04 -4.89072062e+03 -9.76499561e+03
   4.39100093e+03 -2.07868824e+04 -9.62283190e+03  1.24296818e+04
  -1.02994294e+04  3.27586765e+04  2.21873290e+04 -9.42676168e+03
  -3.44689099e+03 -7.08017223e+03  3.15109761e+03  1.42124986e+04
   3.04032454e+04 -8.53254

Regularization imposes a penalty on having a lot of large weights. Notice how some of our ridge regression weights dropped to zero. Let's also look at the sum of our weights before and after regularization:

In [75]:
print("Original Sum: {}".format(round(sum(reg.coef_[0]), 1)))
print("Regularized Sum: {}".format(round(sum(clf.coef_[0]),1)))

Original Sum: -424342.2
Regularized Sum: -143685.1


So let's count the ways this was a bad model:
1. We didn't test any assumptions of normality
2. No checks of colinearity- can affect model performance
3. Are the numeric features meaningful? i.e. Just because MSSubClass is a number (60, 70, 80...), doesn't mean math operations on those numbers are valid.
4. We have a LOT of columns given how few rows of data we have, and usually want 100 rows per column. We're about 21x short.

In [93]:
y_score = copy.deepcopy(y_test)
y_score['Pred_clf'] = clf.predict(X_test)
y_score.head()
# Calculate RMSE according to Kaggle method (RMSE between log-pred and log-act)

,SalePrice,Pred_clf
412,222000.0,302309.723031
1280,227000.0,206856.466386
802,189000.0,188727.422479
331,139000.0,133845.233805
635,200000.0,183517.046195
